In [1]:
! pip install transformers==4.23.1
! pip install torch torchvision
! pip install tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 44.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 85.0 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.0
    Uninstalling tokenizers-0.15.0:
      Successfully uninstalled tokenizers-0.15.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


In [2]:
from transformers import AutoModel, AutoTokenizer
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F
from transformers import BertTokenizer, BertForSequenceClassification
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from transformers import AdamW
# from GPUtil import showUtilization as gpu_usage
# from numba import cuda
import torch
from transformers import BertTokenizer, BertModel
from torch.utils.data import DataLoader, TensorDataset

In [3]:
print("CUDA Available: ", torch.cuda.is_available())

CUDA Available:  True


In [4]:
torch.cuda.empty_cache()
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [5]:
device

device(type='cuda', index=0)

In [6]:
file_path = 'CHATGPT_YOUTUBE_dataset_answers_all_dataset.csv'
df = pd.read_csv(file_path)
train_df = df[df['exp_split'] == 'train']
test_df = df[df['exp_split'] == 'test']

In [7]:
print("Train df: ", len(train_df))
print("Test_df: ", len(test_df))

Train df:  1024
Test_df:  4028


In [8]:
import gc
# del variables
gc.collect()

59

# Trying with aggregation

In [10]:
# Taking Bert Base Uncased model for tokenizing additional features

In [11]:
from transformers import AutoModel, AutoTokenizer

# Additional Projection Layer to map the size from 1024 to 512

In [12]:
file_path = 'CHATGPT_YOUTUBE_dataset_answers_all_dataset.csv'
df = pd.read_csv(file_path)
train_df = df[df['exp_split'] == 'train']
test_df = df[df['exp_split'] == 'test']

In [13]:
bert_model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained("GroNLP/hateBERT") ## need this for tokenizing the input text in data loader
model_bert = AutoModel.from_pretrained(bert_model_name)
tokenizer_bert = AutoTokenizer.from_pretrained(bert_model_name)

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/151 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [14]:
class AdditionalCustomDataset(Dataset):
  def __init__(self, texts, labels, additional_texts, tokenizer, bert_tokenizer, max_length):
    self.texts = texts
    self.labels = labels
    self.additional_texts = additional_texts
    self.tokenizer = tokenizer
    self.bert_tokenizer = bert_tokenizer
    self.max_length = max_length

  def __len__(self):
    return len(self.texts)

  def __getitem__(self, idx):
    texts = self.texts[idx]
    additional_texts = self.additional_texts[idx]
    labels = self.labels[idx]
    encoding = self.tokenizer(texts, max_length=self.max_length, truncation=True, padding='max_length', return_tensors='pt')
    additional_encoding = self.bert_tokenizer(additional_texts, max_length=self.max_length, truncation=True, padding='max_length', return_tensors='pt')
    original_input_ids = encoding['input_ids'].squeeze()
    additional_input_ids = additional_encoding['input_ids'].squeeze()
    input_ids = torch.cat((encoding["input_ids"], additional_encoding["input_ids"]), dim=1)
    original_attention_mask = encoding['attention_mask'].squeeze()
    additional_attention_mask = additional_encoding['attention_mask'].squeeze()
    attention_mask = torch.cat((encoding["attention_mask"], additional_encoding["attention_mask"]), dim=1)
    labels = labels
    return original_input_ids, original_attention_mask, additional_input_ids, additional_attention_mask, labels
    # return input_ids, attention_mask, labels
    # return encoding, additional_encoding, labels

In [15]:
#Splitting training and validation testing split to test accuracy
train_text, val_texts, train_labels, val_labels = train_test_split(train_df['text'].tolist(),train_df['label'].tolist(), test_size = 0.2, random_state = 42)
add_train_text, add_val_texts, add_train_labels, add_val_labels = train_test_split(train_df['ChatGPT Response'].tolist(),train_df['label'].tolist(), test_size = 0.2, random_state = 42)
# train_text, add_train_texts, val_texts, train_labels, add_train_labels, val_labels = train_test_split(train_df['text'].tolist(), train_df['ChatGPT Response'], train_df['label'].tolist(), test_size = 0.2, random_state = 42)

## Creating a CustomDataset
train_dataset = AdditionalCustomDataset(train_text, train_labels, add_train_text, tokenizer, tokenizer_bert, max_length = 512)
val_dataset = AdditionalCustomDataset(val_texts, val_labels, add_val_texts, tokenizer, tokenizer_bert, max_length = 512)

#Creating dataloader object to train the model
train_dataloader = DataLoader(train_dataset, batch_size=2, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=2, shuffle=True)

# Trying with concatenated input

In [16]:
class ProjectionMLP(nn.Module):
    def __init__(self, input_size, output_size):
        super(ProjectionMLP, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_size, output_size),
            nn.ReLU(),
            nn.Linear(output_size, 2)
        )

    def forward(self, x):
        return self.layers(x)

In [17]:
class ConcatModel(nn.Module):
    def __init__(self, hatebert_model, additional_model, projection_mlp, freeze_additional_model=True):
        super(ConcatModel, self).__init__()
        self.hatebert_model = hatebert_model
        self.additional_model = additional_model
        self.projection_mlp = projection_mlp

        if freeze_additional_model:
            for param in self.additional_model.parameters():
                param.requires_grad = False


    def forward(self, input_ids, attention_mask, additional_input_ids, additional_attention_mask):
        # Forward pass through the HateBERT model
        hatebert_outputs = self.hatebert_model(input_ids=input_ids, attention_mask=attention_mask)
        hatebert_embeddings = hatebert_outputs.last_hidden_state[:, 0, :]  # Assuming [CLS] token representation

        # Forward pass through the Additional Model
        additional_outputs = self.additional_model(input_ids=additional_input_ids, attention_mask=additional_attention_mask)
        additional_embeddings = additional_outputs.last_hidden_state[:, 0, :]  # Assuming [CLS] token representation

        # Concatenate the embeddings
        concatenated_embeddings = torch.cat((hatebert_embeddings, additional_embeddings), dim=1)
        # print("Size of concatenated embeddings:", concatenated_embeddings.size())

        # Project concatenated embeddings
        projected_embeddings = self.projection_mlp(concatenated_embeddings)

        return projected_embeddings

In [18]:
hatebert_model = BertModel.from_pretrained("GroNLP/HateBERT")
additional_model = BertModel.from_pretrained("bert-base-uncased")
projection_mlp = ProjectionMLP(input_size=1536, output_size=512)

concat_model = ConcatModel(hatebert_model=hatebert_model, additional_model=additional_model, projection_mlp=projection_mlp, freeze_additional_model=True)


config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at GroNLP/HateBERT were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_rel

In [19]:
concat_model

ConcatModel(
  (hatebert_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [20]:
concat_model = concat_model.to(device)

In [21]:
optimizer = AdamW(concat_model.parameters(), lr=2e-5)
criterion = nn.CrossEntropyLoss()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [22]:
from tqdm import tqdm

num_epochs = 3  # 5 is too much, overfits
for epoch in range(num_epochs):
    concat_model.train()

    train_losses = []
    train_accuracy = 0
    train_epoch_size = 0

    with tqdm(train_dataloader, desc=f'Epoch {epoch + 1}', dynamic_ncols=True) as loop:
        for batch in loop:
            input_ids, attention_mask, additional_input_ids, additional_attention_mask, labels = batch

            if torch.cuda.is_available():
                input_ids = input_ids.to(device)
                attention_mask = attention_mask.to(device)
                additional_input_ids = additional_input_ids.to(device)
                additional_attention_mask = additional_attention_mask.to(device)
                labels = labels.to(device)

            # Forward pass through the ConcatModel
            optimizer.zero_grad()
            outputs = concat_model(input_ids=input_ids, attention_mask=attention_mask, additional_input_ids=additional_input_ids, additional_attention_mask=additional_attention_mask)
            loss = criterion(outputs, labels)
            loss = criterion(outputs.view(-1, 2), labels.view(-1)) # 2 is number of labels

            # Backward pass and optimization
            loss.backward()
            optimizer.step()

            train_losses.append(loss.item())

            # Update accuracy and epoch size
            predictions = torch.argmax(outputs, dim=1)
            train_accuracy += (predictions == labels).sum().item()
            train_epoch_size += len(labels)

            # Update tqdm progress bar with set_postfix
            loop.set_postfix(loss=loss.item(), accuracy=train_accuracy / train_epoch_size)

    # Evaluation on the validation set
    concat_model.eval()

    val_predictions = []
    val_labels = []

    with torch.no_grad(), tqdm(val_dataloader, desc='Validation', dynamic_ncols=True) as loop:
        for batch in loop:
            input_ids, attention_mask, additional_input_ids, additional_attention_mask, labels = batch

            if torch.cuda.is_available():
                input_ids = input_ids.to(device)
                attention_mask = attention_mask.to(device)
                additional_input_ids = additional_input_ids.to(device)
                additional_attention_mask = additional_attention_mask.to(device)
                labels = labels.to(device)

            # Forward pass through the ConcatModel
            outputs = concat_model(input_ids=input_ids, attention_mask=attention_mask, additional_input_ids=additional_input_ids, additional_attention_mask=additional_attention_mask)
            sm = nn.Softmax(dim=1)
            predictions = torch.argmax(sm(outputs), dim=1)
            # print("prediction: ", predictions)
            # sm = nn.Softmax(dim=1)
            # predictions2 = sm(outputs)
            # print("prediction2: ", predictions2)
            val_predictions.extend(predictions.cpu().numpy())
            val_labels.extend(labels.cpu().numpy())

    # Calculate and print validation accuracy
    accuracy = accuracy_score(val_predictions, val_labels)
    print(f"Epoch {epoch + 1}: Validation Accuracy: {accuracy:.4f}, Avg. Train Loss: {sum(train_losses) / len(train_losses):.4f}")


Validation: 100%|██████████| 103/103 [00:14<00:00,  7.00it/s]


Epoch 1: Validation Accuracy: 0.8585, Avg. Train Loss: 0.5223


Validation: 100%|██████████| 103/103 [00:14<00:00,  6.91it/s]


Epoch 2: Validation Accuracy: 0.8293, Avg. Train Loss: 0.1797


Validation: 100%|██████████| 103/103 [00:14<00:00,  6.89it/s]

Epoch 3: Validation Accuracy: 0.8439, Avg. Train Loss: 0.0745


In [ ]:
torch.save(concat_model, 'combined_model_fine_tuned.pt')

In [ ]:
# Testing Accuracies

In [23]:
test_texts = test_df['text'].tolist()
add_test_texts = test_df['ChatGPT Response'].tolist()
test_labels = test_df['label'].tolist()

In [24]:
test_dataset = AdditionalCustomDataset(train_text, train_labels, add_train_text, tokenizer, tokenizer_bert, max_length = 512)
test_dataloader = DataLoader(test_dataset, batch_size=2, shuffle=True)

In [25]:
concat_model.eval()
test_predictions = []
test_labels = []
with torch.no_grad():
    for batch in test_dataloader:
        input_ids, attention_mask, additional_input_ids, additional_attention_mask, labels = batch
        if torch.cuda.is_available():
                input_ids = input_ids.to(device)
                attention_mask = attention_mask.to(device)
                additional_input_ids = additional_input_ids.to(device)
                additional_attention_mask = additional_attention_mask.to(device)
                labels = labels.to(device)
        outputs = concat_model(input_ids=input_ids, attention_mask=attention_mask, additional_input_ids=additional_input_ids, additional_attention_mask=additional_attention_mask)
        predictions = torch.argmax(outputs, dim=1)
        test_predictions.extend(predictions.cpu().numpy())
        test_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(test_predictions, test_labels)

print("Accuracy of test dataset:", accuracy)

Accuracy of test dataset: 0.9963369963369964


In [ ]:
torch.cuda.empty_cache()
import gc
gc.collect()

84

In [ ]:
del concat_model